In [1]:
import glob
import json

import pandas as pd

from pathlib import Path

from experiment import _recursive_split


In [2]:
#Adjust Paths according to storage folder

glob_a = str(Path(f"../../Ergebnisse Imputation Pascal/results_complete") / "**" / "**" / "MCAR" / "**" / "single_single" / "elapsed_train_time_*.json")
glob_b = str(Path(f"../../Ergebnisse Imputation Pascal/results_complete") / "**" / "**" / "MAR" / "**" / "single_single" / "elapsed_train_time_*.json")
glob_c = str(Path(f"../../Ergebnisse Imputation Pascal/results_complete") / "**" / "**" / "MNAR" / "**" / "single_single" / "elapsed_train_time_*.json")

training_time_files = [*glob.glob(glob_a, recursive=True), *glob.glob(glob_b, recursive=True), *glob.glob(glob_c, recursive=True)]


In [3]:
training_time_files

['..\\..\\Ergebnisse Imputation Pascal\\results_complete\\corrupted_binary_experiment\\AutoKerasImputer\\1046\\MCAR\\0.01\\single_single\\elapsed_train_time_end.json',
 '..\\..\\Ergebnisse Imputation Pascal\\results_complete\\corrupted_binary_experiment\\AutoKerasImputer\\1046\\MCAR\\0.1\\single_single\\elapsed_train_time_end.json',
 '..\\..\\Ergebnisse Imputation Pascal\\results_complete\\corrupted_binary_experiment\\AutoKerasImputer\\1046\\MCAR\\0.3\\single_single\\elapsed_train_time_end.json',
 '..\\..\\Ergebnisse Imputation Pascal\\results_complete\\corrupted_binary_experiment\\AutoKerasImputer\\1046\\MCAR\\0.5\\single_single\\elapsed_train_time_end.json',
 '..\\..\\Ergebnisse Imputation Pascal\\results_complete\\corrupted_binary_experiment\\AutoKerasImputer\\1120\\MCAR\\0.01\\single_single\\elapsed_train_time_fAlpha_.json',
 '..\\..\\Ergebnisse Imputation Pascal\\results_complete\\corrupted_binary_experiment\\AutoKerasImputer\\1120\\MCAR\\0.1\\single_single\\elapsed_train_time_fAl

In [4]:
column_names = ["imputer", "task", "type", "fraction"]

dfs = []

for path in training_time_files:
    df = pd.read_json(path, orient="index").T.reset_index(drop=True)
    df[column_names] = _recursive_split(path)[5:-2]
    dfs.append(df)

training_time_all = pd.concat(dfs)

In [5]:
training_time_all

,mean,std,imputer,task,type,fraction
0,859.673591,105.795643,AutoKerasImputer,1046,MCAR,0.01
0,749.567607,129.688023,AutoKerasImputer,1046,MCAR,0.1
0,1182.058795,387.517020,AutoKerasImputer,1046,MCAR,0.3
0,940.145555,324.626005,AutoKerasImputer,1046,MCAR,0.5
0,1010.690768,196.128704,AutoKerasImputer,1120,MCAR,0.01
...,...,...,...,...,...,...
0,13.928034,0.403496,VAEImputer,42688,MNAR,0.5
0,17.381353,1.654779,VAEImputer,42712,MNAR,0.01
0,19.340768,4.636729,VAEImputer,42712,MNAR,0.1
0,18.814621,3.326702,VAEImputer,42712,MNAR,0.3


In [6]:
training_time = training_time_all.drop("std", axis=1)
training_time = training_time.rename(columns={"mean": "training_time"})
training_time = training_time.replace({
    "ModeImputer": "Mean/Mode",
    "KNNImputer": "$k$-NN",
    "ForestImputer": "Random Forest",
    "AutoKerasImputer": "Discriminative DL",
    "VAEImputer": "VAE",
    "GAINImputer": "GAIN"    
})

In [7]:
# First calculate mean and relative std for each imputer and task
training_time_grouped = training_time.groupby(["imputer", "task"]).agg(["mean", "std"])
training_time_grouped.columns = ["mean", "std"]
training_time_grouped["rel std"] = training_time_grouped.loc[:, "std"] /  training_time_grouped.loc[:, "mean"]

# Then average over all data sets. This leads to relativ sd that is less dependant on the data set size.
training_time_grouped = training_time_grouped.groupby("imputer").agg(["mean", "std"])
training_time_grouped = training_time_grouped.loc[:, [("mean", "mean"), ("rel std", "mean")]]

C:\Users\dittr\AppData\Local\Temp\ipykernel_6608\1497049135.py:2: FutureWarning: ['type', 'fraction'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  training_time_grouped = training_time.groupby(["imputer", "task"]).agg(["mean", "std"])


# Predict Time

In [8]:
#Adjust Paths according to storage folder

subset_a = str(Path(f"../../Ergebnisse Subset") / "**" / "**" / "MCAR" / "**" / "single_single" / "elapsed_train_time_*.json")
subset_b = str(Path(f"../../Ergebnisse Subset") / "**" / "**" / "MCAR" / "**" / "single_single" / "elapsed_train_time_*.json")
subset_c = str(Path(f"../../Ergebnisse Subset") / "**" / "**" / "MCAR" / "**" / "single_single" / "elapsed_train_time_*.json")


subset_training_time_files = [*glob.glob(subset_a, recursive=True), *glob.glob(subset_b, recursive=True), *glob.glob(subset_c, recursive=True)]


In [10]:
column_names = ["imputer", "task", "type", "fraction"]

dfs = []

for path in subset_training_time_files:
    df = pd.read_json(path, orient="index").T.reset_index(drop=True)
    df[column_names] = _recursive_split(path)[4:-2]
    dfs.append(df)

predict_time_all = pd.concat(dfs)

In [11]:
predict_time_all

,mean,std,imputer,task,type,fraction
0,189.353443,11.328755,AutoKerasImputer,1046,MCAR,0.01
0,160.958650,7.555858,AutoKerasImputer,1046,MCAR,0.3
0,203.497555,6.535599,AutoKerasImputer,1120,MCAR,0.01
0,189.612691,11.005251,AutoKerasImputer,1120,MCAR,0.1
0,177.747713,10.570863,AutoKerasImputer,1120,MCAR,0.3
...,...,...,...,...,...,...
0,4.785595,0.193312,VAEImputer,42688,MCAR,0.5
0,6.210568,0.512301,VAEImputer,42712,MCAR,0.01
0,6.051440,0.510086,VAEImputer,42712,MCAR,0.1
0,6.036757,0.348287,VAEImputer,42712,MCAR,0.3


In [12]:
predict_time = predict_time_all.drop("std", axis=1)
predict_time = predict_time.rename(columns={"mean": "subset_training_time"})
predict_time = predict_time.replace({
    "ModeImputer": "Mean/Mode",
    "KNNImputer": "$k$-NN",
    "ForestImputer": "Random Forest",
    "AutoKerasImputer": "Discriminative DL",
    "VAEImputer": "VAE",
    "GAINImputer": "GAIN"    
})

In [13]:
# First calculate mean and relative std for each imputer and task
predict_time_grouped = predict_time.groupby(["imputer", "task"]).agg(["mean", "std"])
predict_time_grouped.columns = ["mean", "std"]
predict_time_grouped["rel std"] = predict_time_grouped.loc[:, "std"] /  predict_time_grouped.loc[:, "mean"]

# Then average over all data sets. This leads to relativ sd that is less dependant on the data set size.
predict_time_grouped = predict_time_grouped.groupby("imputer").agg(["mean", "std"])
predict_time_grouped = predict_time_grouped.loc[:, [("mean", "mean"), ("rel std", "mean")]]
predict_time_grouped = predict_time_grouped.loc[["Mean/Mode", "$k$-NN", "Random Forest", "Discriminative DL", "VAE", "GAIN"],:].reset_index()

C:\Users\dittr\AppData\Local\Temp\ipykernel_6608\2565147978.py:2: FutureWarning: ['type', 'fraction'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  predict_time_grouped = predict_time.groupby(["imputer", "task"]).agg(["mean", "std"])


In [14]:
predict_time_grouped

,imputer,mean,rel std
,,mean,mean
0,Mean/Mode,0.000329,0.038859
1,$k$-NN,0.148897,1.015851
2,Random Forest,0.609447,0.417495
3,Discriminative DL,198.556602,0.101573
4,VAE,6.183761,0.058060
5,GAIN,76.970171,0.043085


# Latex Table

In [15]:
table_latex = training_time_grouped.loc[["Mean/Mode", "$k$-NN", "Random Forest", "Discriminative DL", "VAE", "GAIN"],:].reset_index()
table_latex[("a", "mean")] = predict_time_grouped[("mean", "mean")]
table_latex[("a", "std")] = predict_time_grouped[("rel std", "mean")]
table_latex = pd.DataFrame(table_latex.values, columns=pd.MultiIndex.from_tuples([("Imputation Method", ""), ("Training", "Mean Duration"), ("Training", "Rel. SD"), ("Subset", "Mean Duration"), ("Subset", "Rel. SD")]))
table_latex

Imputation Method      Training                  Subset          
                     Mean Duration   Rel. SD Mean Duration   Rel. SD
0          Mean/Mode      0.000476  0.058418      0.000329  0.038859
1             $k$-NN      1.081538  0.700514      0.148897  1.015851
2      Random Forest     17.588743  0.271258      0.609447  0.417495
3  Discriminative DL   1018.135918  0.255103    198.556602  0.101573
4                VAE     18.289359   0.08272      6.183761   0.05806
5               GAIN    182.681209  0.120072     76.970171  0.043085

In [16]:
print(
    table_latex.to_latex(
        caption="Training and inference duration for each imputation method in seconds. We use the wall-time to measure the durations for training including hyperparameter optimization and inference for MCAR missingness pattern and all missingness fractions shown in Table TODO. Because training and inference durations depend heavily on the data set, we first average all measurements for imputation method and data set combinations and calculate the standard deviation relatives relative to there mean durations. Second, we average both mean durations and relative standard devaition for the imputation methods. Abbreviations: Rel. SD means Relative Standard Deviation.",
        label="tab:time",
        index=False,
        escape=False
    )
)

\begin{table}
\centering
\caption{Training and inference duration for each imputation method in seconds. We use the wall-time to measure the durations for training including hyperparameter optimization and inference for MCAR missingness pattern and all missingness fractions shown in Table TODO. Because training and inference durations depend heavily on the data set, we first average all measurements for imputation method and data set combinations and calculate the standard deviation relatives relative to there mean durations. Second, we average both mean durations and relative standard devaition for the imputation methods. Abbreviations: Rel. SD means Relative Standard Deviation.}
\label{tab:time}
\begin{tabular}{lllll}
\toprule
Imputation Method & \multicolumn{2}{l}{Training} & \multicolumn{2}{l}{Subset} \\
                  & Mean Duration &   Rel. SD & Mean Duration &   Rel. SD \\
\midrule
        Mean/Mode &      0.000476 &  0.058418 &      0.000329 &  0.038859 \\
           $k$-NN

C:\Users\dittr\AppData\Local\Temp\ipykernel_6608\3571645291.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  table_latex.to_latex(
